# K-Nearest Neighbor Lab





In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import matplotlib.pyplot as plt

## 1. (40%) Implement the k-nearest neighbor algorithm and the k-nearest neighbor regression algorithm, including optional distance weighting for both algorithms. 

### Code requirements
- Use Euclidean distance to decide closest neighbors. 


In [ ]:
class KNNClassifier(BaseEstimator,ClassifierMixin):


    def __init__(self, columntype=[], weight_type='inverse_distance', k=5): ## add parameters here
        """
        Args:
            columntype for each column tells you if continues[real] or if nominal[categoritcal].
            weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
        """
        self.columntype = columntype #Note This won't be needed until part 5
        self.weight_type = weight_type
        self.k = k

    def fit(self, data, labels):
        """ Fit the data; run the algorithm (for this lab really just saves the data :D)
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        self.data = data
        self.labels = labels
        return self
    
    def predict(self, data):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """

        predict_array = []
        for i, i_point in enumerate(data):
          distances = []
          for j, j_point in enumerate(self.data):
            distances.append(((np.linalg.norm(j_point - i_point)), self.labels[j]))
          distances.sort()
          neighbors = distances[:self.k]

          target_classes = np.unique(self.labels)
          votes = dict.fromkeys(target_classes, 0)
          if self.weight_type == 'inverse_distance':
            for n in neighbors:
              votes[n[1]] += 1/n[0]**2 # Vote is weighted by distance
          else:
            for n in neighbors:
              votes[n[1]] += 1
          predict_class = max(votes, key=votes.get)
          predict_array.append(predict_class)

        return predict_array

    #Returns the Mean score given input data and labels
    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets
        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """
        count = 0
        predictions = self.predict(X)
        for i in range(len(predictions)):
          if predictions[i] == y[i]:
            count += 1
        mean = count / len(predictions)

        return mean
    

## 1.1 Debug
Debug your model by running it on the [seismic bumps](https://archive.ics.uci.edu/ml/datasets/seismic-bumps) problem.
- Use this [training set](https://github.com/cs472ta/CS472/blob/master/datasets/seismic-bumps_train.arff) and this [test set](https://github.com/cs472ta/CS472/blob/master/datasets/seismic-bumps_test.arff)
- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance

---

Expected Results:
- Acc = [93.57]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/seismic-bump-prediction.csv)


In [ ]:
# Load seismic bumps data

from scipy.io import arff
import pandas as pd
from sklearn import preprocessing

!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff --output seismic-train.arff
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_test.arff --output seismic-test.arff

seismic_train = arff.loadarff('seismic-train.arff')
seismic_test = arff.loadarff('seismic-test.arff')

seismic_train = pd.DataFrame(seismic_train[0])
seismic_test = pd.DataFrame(seismic_test[0])

seismic_train['Class'] = [int(s.decode()) for s in seismic_train['Class']]
seismic_test['Class'] = [int(s.decode()) for s in seismic_test['Class']]

seismic_train.head()

seismic_train = np.array(seismic_train)
seismic_train_data = seismic_train[:,:-1]
seismic_train_targets = seismic_train[:,-1]

seismic_test = np.array(seismic_test)
seismic_test_data = seismic_test[:,:-1]
seismic_test_targets = seismic_test[:,-1]

# Train on training set

my_knn = KNNClassifier(weight_type='no_weight', k=3)
my_knn.fit(seismic_train_data, seismic_train_targets)

# Predict on test set

my_knn.predict(seismic_test_data)
my_knn.score(seismic_test_data, seismic_test_targets)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6392  100  6392    0     0   122k      0 --:--:-- --:--:-- --:--:--  120k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9597  100  9597    0     0   153k      0 --:--:-- --:--:-- --:--:--  153k


0.9357142857142857

## 2. (10%) Use the k-nearest neighbor algorithm (without distance weighting) for the [magic telescope](http://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope) problem

- Use this [training set](https://github.com/cs472ta/CS472/blob/master/datasets/magic_telescope_train.arff) and this [test set](https://github.com/cs472ta/CS472/blob/master/datasets/magic_telescope_test.arff) 

## 2.1
- Try it with k=3 with normalization (input features normalized between 0 and 1) and without normalization and discuss the accuracy results on the test set. Use the formula (x-xmin)/(xmax-xmin)



In [ ]:
# Load magic telescope data
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff --output magic_telescope_train.arff
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff --output magic_telescope_test.arff

train_data = arff.loadarff('magic_telescope_train.arff')
test_data = arff.loadarff('magic_telescope_test.arff')

magic_train_df = pd.DataFrame(train_data[0])
magic_test_df = pd.DataFrame(test_data[0])

magic_train_df['class'] = magic_train_df['class'].str.decode('utf-8')
magic_test_df['class'] = magic_test_df['class'].str.decode('utf-8')

magic_train = np.array(magic_train_df)
magic_train_data = magic_train[:,:-1]
magic_train_targets = magic_train[:,-1]

magic_test = np.array(magic_test_df)
magic_test_data = magic_test[:,:-1]
magic_test_targets = magic_test[:,-1]

maximums = np.amax(magic_train_data, axis=0)
minimums = np.amin(magic_train_data, axis=0)
  
magic_test_data_normalized = np.zeros_like(magic_test_data)

for i in range(len(magic_test_data[0])):
  for j in range(len(magic_test_data)):
    magic_test_data_normalized[j][i] = (magic_test_data[j][i] - minimums[i]) / (maximums[i] - minimums[i])

# Train/Predict with normalization

magic_knn = KNNClassifier(k=3)
magic_knn.fit(magic_train_data, magic_train_targets)
norm_score = magic_knn.score(magic_test_data_normalized, magic_test_targets)
print(norm_score)

# Train/Predict without normalization

without_norm_score = magic_knn.score(magic_test_data, magic_test_targets)
print(without_norm_score)



*Discuss the accuracy results*

## 2.2

- With just the normalized training set as your data, graph classification accuracy on the test set with odd values of k from 1 to 15.
    - Which value of k is the best in terms of classification accuracy?
    - As a rough sanity check, typical knn accuracies for the magic telescope data set are 75-85%

In [ ]:
# Train/Predict with normalization using k=1,3,...,15

# Graph classification accuracy over k


# For the rest of the experiments use only normalized data

## 3. (10%) Use the regression variation of your algorithm (without distance weighting) for the [housing price prediction](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) problem.

- Use this [training set](https://github.com/cs472ta/CS472/blob/master/datasets/housing_train.arff) and this [test set](https://github.com/cs472ta/CS472/blob/master/datasets/housing_test.arff).
- Use Mean Square Error (MSE) on the test set as your accuracy metric for this case.
    - Do not normalize regression output values
- Graph MSE on the test set with odd values of k from 1 to 15


In [ ]:
# Load housing price prediction data

# Download file with curl
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_train.arff --output housing-train.arff
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_test.arff --output housing-test.arff

housing_train = arff.loadarff('housing-train.arff')
housing_test = arff.loadarff('housing-test.arff')

housing_train = pd.DataFrame(housing_train[0])
housing_test = pd.DataFrame(housing_test[0])

housing_train.CHAS = [int(s.decode()) for s in housing_train.CHAS]
housing_test.CHAS = [int(s.decode()) for s in housing_test.CHAS]

print(housing_train)

housing_train = np.array(housing_train)
housing_train_data = housing_train[:,:-1]
housing_train_targets = housing_train[:,-1]

housing_test = np.array(housing_test)
housing_test_data = housing_test[:,:-1]
housing_test_targets = housing_test[:,-1]

# Train/Predict using k=1,3,...,15

# Graph MSE over k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34205  100 34205    0     0   263k      0 --:--:-- --:--:-- --:--:--  265k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3885  100  3885    0     0  34078      0 --:--:-- --:--:-- --:--:-- 33782
        CRIM   ZN  INDUS  CHAS    NOX  ...    TAX  PTRATIO       B  LSTAT  MEDV
0    0.02731  0.0   7.07     0  0.469  ...  242.0     17.8  396.90   9.14  21.6
1    0.02729  0.0   7.07     0  0.469  ...  242.0     17.8  392.83   4.03  34.7
2    0.03237  0.0   2.18     0  0.458  ...  222.0     18.7  394.63   2.94  33.4
3    0.06905  0.0   2.18     0  0.458  ...  222.0     18.7  396.90   5.33  36.2
4    0.02985  0.0   2.18     0  0.458  ...  222.0     18.7  394.12   5.21  28.7
..       ...  ...    ...   ...    ...  ...    

## 4. (15%) Repeat your experiments for magic telescope and housing using distance-weighted (inverse of distance squared) voting and discuss your results.


In [ ]:
# Train/Predict magic telescope using distance-weighted voting

# Train/Predict housing using distance-weighted voting


*Discuss your results*

## 5. (10%) Use the k-nearest neighbor algorithm to solve the [credit-approval](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) (credit-a) problem.

- Use this [dataset](https://github.com/cs472ta/CS472/blob/master/datasets/credit_approval.arff.txt)
- Note that this set has both continuous and nominal attributes, together with don’t know values. 
- Implement and justify a distance metric which supports continuous, nominal, and don’t know attribute values
    - You need to handle don't knows with the distance metric, not by imputing a value.
    - More information on distance metrics can be found [here](http://axon.cs.byu.edu/~martinez/classes/478/slides/IBL.pdf).
- Use your own choice for k, training/test split, etc.
- As a rough sanity check, typical k-nn accuracies for the credit data set are 70-80%.


In [ ]:
# Load dataset and split into train/test sets

# Train/Predict credit-approval


*Justify your distance metric*

## 6. (15%) Use the scikit's KNN classifier on magic telescope and housing and compare your results.

- Try out different hyperparameters to see how well you can do. 
- Find a data set of your choice (not one previously used) and use the SK version to get results for it.


In [ ]:
# Train/Predict magic telescope using scikit's KNN

from sklearn.neighbors import KNeighborsClassifier

magic_knn = KNeighborsClassifier(n_neighbors=5)
magic_knn.fit(magic_train_data, magic_train_targets)
magic_score = magic_knn.score(magic_test_data, magic_test_targets)
print("Magic Score: ", magic_score)

# Train/Predict housing using scikit's KNN

housing_knn = KNeighborsClassifier(n_neighbors=5)
housing_knn.fit(housing_train_data, housing_train_targets)
housing_score = housing_knn.score(housing_test_data, housing_test_targets)
print("Housing Score: ", housing_score)

# Load a dataset of your choice

new_data = pd.read_csv("/content/KNNAlgorithmDataset.csv")
new_data = new_data[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']]

new_data = np.array(new_data)
new_data_data = new_data[:,:-1]
new_data_targets = new_data[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_data_data, new_data_targets, test_size=0.33, random_state=42)

# Train/Predict your dataset using scikit's KNN

new_knn = KNeighborsClassifier(n_neighbors=5)
new_knn.fit(X_train, y_train)
new_score = new_knn.score(X_test, y_test)
print("New Dataset Score: ", new_score)


*Report your comparison*

## 7. (optional 5% extra credit): For the best value of k for any one of the datasets, implement a reduction algorithm that removes data points in some rational way such that performance does not drop too drastically on the test set given the reduced training set.

- Compare your performance on the test set for the reduced and non-reduced versions and give the number (and percentage) of training examples removed from the original training set. How well does your reduction algorithm work?
    - Note that performance for magic telescope is classification accuracy and for housing it is sum squared error.
    - Magic Telescope has about 12,000 instances and if you use a leave one out style of testing for your data set reduction, then your algorithm will run slow since that is n2 at each step.
    - If you wish, you may use a random subset of 2,000 of the magic telescope instances.
    - More information on reduction techniques can be found [here](http://axon.cs.byu.edu/~martinez/classes/478/slides/IBL.pdf).
